In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import json

# Edit the following constants to change the model and responses file
MODEL_NAME = ""
RESULTS_PATH = "../data/results.json"

with open(RESULTS_PATH, "r") as f:
    results = json.load(f)

# Create individual plots for each dimension
dimensions = ['PDI', 'IDV', 'UAI', 'MAS', 'LTO', 'IVR']
hk_means = {'PDI': 0.68, 'IDV': 0.25, 'UAI': 0.29, 'MAS': 0.57, 'LTO': 0.61, 'IVR': 0.17}
colors = {'PDI': '#7596a5', 'IDV': '#166c9c', 'UAI': '#a94b3b', 'MAS': '#2f774f', 'LTO': '#7f7eaa', 'IVR': '#ca863e'}
fig, axes = plt.subplots(2, 3, figsize=(8, 18))  # Arrange plots in a 3x2 grid

# Create a mapping for full names
full_names = {
    'PDI': 'Power Distance Index',
    'IDV': 'Individualism',
    'UAI': 'Uncertainty Avoidance Index',
    'MAS': 'Masculinity',
    'LTO': 'Long-Term Orientation',
    'IVR': 'Indulgence vs. Restraint'
}

for ax, dimension in zip(axes.flatten(), dimensions):
    # Check if dimension exists in results
    if dimension in results:
        y_data = [results[dimension]]
        x_labels = [MODEL_NAME]
        
        sns.violinplot(data=y_data, ax=ax, color=colors[dimension])
        ax.set_xticks(range(len(x_labels)))
        ax.set_xticklabels(x_labels, rotation=45, ha='right')
        ax.set_ylabel(f'High {dimension} Likelihood')
        ax.set_ylim(-0.2, 1.2)
        ax.axhline(0.4, linestyle='--', color='gray')
        ax.axhline(0.6, linestyle='--', color='gray')
        ax.axhline(hk_means[dimension], linestyle='-', color='red', label='HK Mean')
        ax.set_title(f'{full_names[dimension]} ({dimension})')
        ax.legend()

plt.tight_layout()
plt.show()